## Taxi Problem



Install OpenAI gym

In [ ]:
#!pip install gym --upgrade

Load the game environment and render what it looks like

In [ ]:
import gym

env = gym.make("Taxi-v3").env
env.render()

The core gym interface is `env`.

`env.reset`: Resets the environment and returns a random initial state.

`env.step(action)`: Step the environment by one timestep.

`env.render`: Renders one frame of the environment 

In [ ]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Encode the state, so that the taxi is at row 3, column 1 (3,1). The passenger is at location 2, and the destination is at location 0.

In [ ]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

Print out the reward table, where the rows are the actions (south, north, east, west, pickup, dropoff). The columns are (probability, nextstate, reward, done)

In [ ]:
env.P[328]

Below we will try to brute-force our way to solving the problem withould Reinforcement Learning. Because we have a default rewards table for each state `P` we use that to navigate the taxi.

The infinite loop will run until one passenger reaches a destination (one episode). Simply, when the agent gets awarded with 20 points. 

`env.action_space.sample()`: automatically selects one random action from set of all possible actions

In [ ]:

env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
#print_frames(frames)

The end result is not great. The agent takes many steps and makes a lot of dropoffs at incorrect locations to just deliver one passenger to the correct destination. This is due to the agent not 'learning' from past experiences, meaning that it doesn't matter how often we run the simulation, the agent will never be optimised. The agent has no memory of which action is best for each state, which is where Reinforcement Learning can help.

### Q-learning

Q-learning is a simple Reinforcement Learning algorithm which is able to provide the agent with some memory. Q-learning allows the agent to use the environment's rewards to learn, which over time will provide it with the best action to take when in a given state. 

Specifically, for the Taxi environment, the agent will learn from the rewards table `P`. It does this by looking at what rewards will be given for taking an action in a current state, and then updating a Q-value to remember if the action was beneficial in the long run.

The Q-values are stored in a table called Q-table, where the rows represent the states, and the columns represent the actions possible.

First, the Q-table is initialised, with the size being `(number of possible states, number of actions possible)`

In [ ]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

Below, the training algorithm for Q-learning is implemented, it will update the Q-table as the agent explores the environment over many episodes. 

In [ ]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output


graph = False
num_episodes = 100001

#graph for comparision with Cart Pole 
if graph == True:
    num_episodes = 500 #500 for graphing, 100001 for normal

#store how many timesteps per episode
steps = np.zeros(num_episodes)


# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, num_episodes):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if graph == True:
            steps[i] += 1
        #decide whether to pick a random action or use already computed Q-values. 
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        
        #execute the chosen action
        next_state, reward, done, info = env.step(action) 
        
        #store value for previous state, and for current state (after executing action)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        #calculate the maximum Q-value for the current state (after executing action)
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        #update Q-table
        q_table[state, action] = new_value
        
        #penalise the agent if agents drops off / picks up
        if reward == -10:
            penalties += 1
        
        #set the next state and increment the timestep
        state = next_state
        epochs += 1
    
    #print out how many episodes have been run
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Generate Graph

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import math

if graph == True:
    sns.lineplot(range(len(steps)),steps)
    plt.xlabel("Episode")
    plt.ylabel("Steps")
    plt.title("Taxi-Problem")
    plt.show()

The Q-table has been establised over 100,000 episodes. The Q-values for state 328 can be seen below.

In [ ]:
q_table[328]

We can now simply evaluate the performance of the agent after 'learning'. There is no need to explore any actions anymore as we can be sure the agents has learned which action to select next using the best Q-value.

In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        #get the best action to take, by finding the maximum Q-value for a given state.
        action = np.argmax(q_table[state])
        #execute the action
        state, reward, done, info = env.step(action)
        
        #if agent runs into a wall, give it a penalty
        if reward == -10:
            penalties += 1
        
        #increment timestep
        epochs += 1
    
    #update total penalties accumilated, and timesteps
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")